# Measuring the impact of a website redesign the Bayesian way

## 📖 Background
You work for an early-stage startup in Germany. Your team has been working on a redesign of the landing page. The team believes a new design will increase the number of people who click through and join your site. 

They have been testing the changes for a few weeks and now they want to measure the impact of the change and need you to determine if the increase can be due to random chance or if it is statistically significant.

![cover](cover.png)

The figure above captures our result in a single picture. Do keep reading to understand it!

## 💾 The data
The team assembled the following file:

#### Redesign test data
- "treatment" - "yes" if the user saw the new version of the landing page, no otherwise.
- "new_images" - "yes" if the page used a new set of images, no otherwise.
- "converted" - 1 if the user joined the site, 0 otherwise.

The control group is those users with "no" in both columns: the old version with the old set of images.

In [1]:
# Ignore installation error messages below, this is all OK for our purposes.
!pip -qqq --no-color install pymc3 'numpy>=1.15.0,<1.22.0'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pythonwhat 2.23.1 requires asttokens~=1.1.10, but you have asttokens 2.0.8 which is incompatible.
pythonwhat 2.23.1 requires dill~=0.2.7.1, but you have dill 0.3.5.1 which is incompatible.
pythonwhat 2.23.1 requires jinja2~=2.10, but you have jinja2 3.1.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import pymc3 as pm
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('bmh')
# https://viscid-hub.github.io/Viscid-docs/docs/dev/styles/bmh.html
BMH_COLORS = ['#348ABD', '#A60628', '#7A68A6', '#467821']

np.random.seed(44)
print('Numpy version:', np.__version__)
print('PyMC3 version:', pm.__version__)

ContextualVersionConflict: (scipy 1.9.0 (/usr/local/lib/python3.8/dist-packages), Requirement.parse('scipy<1.8.0,>=1.7.3'), {'pymc3'})

In [3]:
df = pd.read_csv('./data/redesign.csv')
df.head()

We first verify that there are no missing values to ensure that our analysis is correct.

In [4]:
df.isnull().any()

For convenience, we convert "yes" to a `1` and "no" to a `0` so that we can compute statistics.

In [5]:
df['treatment'] = df.treatment.map({'yes': 1, 'no': 0})
df['new_images'] = df.new_images.map({'yes': 1, 'no': 0})

Looking at the summary statistics, we see that the dataset is balanced in every way, 50% of the dataset has a `treatment` and 50% does not; similarly for `new_images`. In fact, it is balanced across all four groups that we have:

1. group $A$: has `treatment` and has `new_images`.
2. group $B$: has `treatment`, but no `new_images`.
3. group $C$: no `treatment`, but has `new_images`.
4. the $\mathrm{control}$ group: no `treatment` and no `new_images`.

Also, observe that the conversion rate across all groups have a mean of 11.3% with a sample standard deviation of 31.7% (which is quite large!).

In [6]:
df.describe()

In [7]:
def grouping(row):
    new_landing = row.treatment
    new_images = row.new_images
    if new_landing and new_images:
        return 'A'
    elif new_landing and not new_images:
        return 'B'
    elif not new_landing and new_images:
        return 'C'
    return 'control'

df['group'] = df.apply(grouping, axis=1)

assert (len(df[df.group == "A"]) 
        == len(df[df.group == "B"]) 
        == len(df[df.group == "C"]) 
        == len(df[df.group == "control"]))

print(f'Size of each group: {len(df[df.group == "A"])}')

## 1. Observables: analyzing the observed conversion rates for each group

To get a first picture of the conversion rates of each group, we look at the following normalized countplot. Looking at the "yes"-es, we observe that all the test groups $A$, $B$, $C$ seems to improve over the control group. In particular, we see that group $B$ seems to improve the most since it has the highest "yes" to "no" ratio i.e. the highest conversion rate.

In [8]:
_, ax = plt.subplots(figsize=(10, 7))

conversion_count = (df.groupby(['group'])['converted']
                    .value_counts(normalize=True)
                    .rename('percentage')
                    .reset_index())

sns.barplot(data=conversion_count, x='converted', y='percentage', hue='group', ax=ax)

ax.set_xlabel('Converted?')
ax.set(xticklabels=['no', 'yes']);

The actual numbers agree with our eye observation. Group $B$ has an observed conversion rate of 12% which is 1.3% more than 10.7%, the observed conversion rate of the control group. Group $A$ and group $C$ on the other hand have approximately a 0.7% and 0.6% increase in its observed conversion rate as compared to the control group's.

In [9]:
GROUP_CONV_RATE = {}

print('Observed conversion rate')
for i in range(15):
    print('-', end='')
print()
for group in df.group.unique():
    conv_rate = df[df.group == group].converted.mean()
    GROUP_CONV_RATE[group] = conv_rate
    print(f'Group {group}: {conv_rate:.3f}')

If you like pivot tables, we can also do that and get the same answer by recalling what the groups $A$, $B$, $C$ and $\mathrm{control}$ means.

In [10]:
pd.pivot_table(df,
               values='converted', 
               index=['treatment'], 
               columns=['new_images'],
               aggfunc=np.mean)

## 2. Measuring success of redesign using Bayesian A/B testing

We will use Bayesian inference to explain the increase in observed conversion rate of groups $A$, $B$, and $C$ since it has better explainability (among many other advantages) as compared to the frequentist approach using p-values. 

We first define a helper function `from_posterior` which helps us to update our "prior" since our dataset is quite large to do Bayesian inference on a small machine.

In [11]:
from pymc3.distributions import Interpolated

def from_posterior(param, samples):
    """https://docs.pymc.io/en/v3/pymc-examples/examples/pymc3_howto/updating_priors.html"""
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, 100)
    y = stats.gaussian_kde(samples)(x)

    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    return Interpolated(param, x, y)

Below, we extract the group `converted` observations which will be used for inference in the next section.

In [12]:
def prep_group_data(group_name):
    return df[df.group == group_name].converted.to_numpy().reshape(-1, 1)

group_data = {
    'A': prep_group_data('A'),
    'B': prep_group_data('B'),
    'C': prep_group_data('C'),
    'control': prep_group_data('control'),
}

### Thinking how our data is generated

There are four unknown parameters to infer here – these are the conversion rates of the groups $A$, $B$, $C$, and $\mathrm{control}$ which are probabilities between 0 and 1. We will label these conversion rates as $p_A$, $p_B$, $p_C$, and $p_{\mathrm{control}}$ respectively. 

Since we do not have any prior belief on these conversion rates, we assume as a starting point that each of these conversion rates is equally likely over all possible values between 0 and 1. That is, for example, our current initial belief is that it is equally likely for $p_A$ to be 0.13 as it is to be 0.97 (these are just random numbers). In Bayesian lingo, this means that we assume a uniform prior over the interval $[0, 1]$ for each of these conversion rates. 

We then incorporate our observed conversion rate using a Bernoulli distribution conditioned on the prior conversion rates. This is where "training" happens if we think of Bayesian modelling as training a generic machine learning model. By incorporating observations, we are able to update our prior beliefs into a so-called posterior belief.

In [13]:
def sample_posterior(data, batch_size=5000):
    traces = []
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i+batch_size]
        model = pm.Model()
        with model:
            # Prior
            if len(traces) == 0:
                p = pm.Uniform('p', 0, 1)
            else:
                p = from_posterior('p', burned_trace['p'])
            
            # Likelihood
            obs = pm.Bernoulli('obs', p, observed=batch_data)
        
            # Inference
            trace = pm.sample(11000, step=pm.Metropolis())
            burned_trace = trace[1000:]
            traces.append(burned_trace)
    return traces

Using the Metropolis inference engine, we sample the posterior distribution of the conversion rates $p_A$, $p_B$, $p_C$ and $p_{\mathrm{control}}$ which in probabilistic programming lingo is called *traces*.

In [14]:
import logging
logger = logging.getLogger('pymc3')
logger.setLevel(logging.ERROR)

traces_A = sample_posterior(group_data['A'])
traces_B = sample_posterior(group_data['B'])
traces_C = sample_posterior(group_data['C'])
traces_control = sample_posterior(group_data['control'])

In [15]:
def final_samples(trace):
    return trace[-1]['p']

final_trace = {
    'A': final_samples(traces_A),
    'B': final_samples(traces_B),
    'C': final_samples(traces_C),
    'control': final_samples(traces_control),
}

We plot the posterior distribution of the four conversion rates we wanted to infer:

In [16]:
_, ax = plt.subplots(4, 1, figsize=(10, 10))

ax[0].set_title('Posterior distributions of conversion rates $p_A$, $p_B$, $p_C$ and $p_{\mathrm{control}}$')

for i, group in enumerate(df.group.unique()):
    ax[i].set_xlim(0.08, 0.15)
    
    group_final_trace = final_trace[group]
    label = 'posterior of $p_{\mathrm{' + f'{group}' + '}}$'
    ax[i].hist(group_final_trace, 
               histtype='stepfilled',
               bins=100, alpha=0.85, density=True, 
               label=label, color=BMH_COLORS[i])
    ax[i].vlines(group_final_trace.mean(), 0, 150, color='black', linestyle='--', label='mean')
    ax[i].legend()

plt.show()

To see it more clearly, we plot the difference $\Delta_A$ (respectively $\Delta_B$, $\Delta_C$) between the posterior of the test group $A$ (respectively $B$, $C$) and the posterior of the control group.

In [17]:
_, ax = plt.subplots(3, 1, figsize=(10, 10))

ax[0].set_title('Posterior distributions of conversion rate differences $\Delta_A$, $\Delta_B$, $\Delta_C$')

for i, group in enumerate(['A', 'B', 'C']):
    ax[i].set_xlim(-0.02, 0.04)
    
    group_delta_trace = final_trace[group] - final_trace['control']
    label = f'posterior of $\Delta_{group}$'
    ax[i].hist(group_delta_trace, 
               histtype='stepfilled',
               bins=100, alpha=0.85, density=True, 
               label=label, color=BMH_COLORS[i])
    ax[i].vlines(group_delta_trace.mean(), 0, 150, color='black', linestyle='--', label='mean')
    ax[i].vlines(0, 0, 150, color='black', alpha=0.2)
    ax[i].legend()

plt.show()

Based on the posterior distributions of conversion rate differences $\Delta_A$, $\Delta_B$, $\Delta_C$, we observe that most of the distribution lies above 0. This implies that each group $A$, $B$ and $C$ is more likely to have a higher conversion rate than the control group. That is, the corresponding website version of groups $A$, $B$ and $C$ respectively are all major improvements over the current version. However, it should be obvious that there's a clear winner: group $B$ seems to be most likely to have a higher conversion rate than the control. 

The actual probabilities of the group are computed below:

In [18]:
for group in ['A', 'B', 'C']:
    print('Probability that user group {0} has higher conversion rate than control: {1:.3f}'\
          .format(group, ((final_trace[group] - final_trace['control']) > 0).mean()))

### Conclusion

The probabilities agree with our observation and discussion. In fact, we have a more granular observation.

Group $B$ with approximately 0.99 probability is the most likely to have a higher conversion rate than the control. Now group $A$ has a probability of 0.93 which is higher than the probability of group $C$ of 0.9. So it is more likely that user group $A$ has a higher conversion rate than the control, as compared to group $C$. This establishes a hierarchy of website redesign performance as follows:

$B$ is best peforming, followed by $A$, and then lastly $C$.

## 3. Fresh new landing page, but same good ol' images is the way to go

Recall what the group $A$, $B$ and $C$ means:

- user group $A$: saw the **new** version of the **landing page**, with **new set of images**.
- user group $B$: saw the **new** version of the **landing page**, with **old set of images**. 
- user group $C$: saw the **old** version of the **landing page**, with **new set of images**.
- $\mathrm{control}$ user group: saw the **old** version of **both landing page and set of images**.

So our conclusion from the previous section suggested that there is a hierarchy of website redesign performance like so:

1. **Best performing**: use the new landing page, but keep the old set of images;
2. **Second best**: use both the new landing page and new set of images;
3. **Third best**: use the new set of images, but keep the old landing page.

And each of them have more than 90% probability of being better than just using the old landing page and old set of images! So we only have a 10% chance of being wrong if we choose any of them, which is quite a small risk already.

### Conclusion

But if we need to choose one, just redesign the website using the **new landing page** but maintain the **old set of images**. In this case, we only have a 1% chance of being wrong, a very very tiny risk.